# Minimal models

We select models with either very similar opinions, or very different opinions, and have them debate political issues. 

We expect those that disagree on a issue to become more upset. 

The model needs to capture some reasonable dynamics of human interaction. 

We assert that if a pro-choice strong Democrat and a pro-life strong Republican discuss abortion, it will be less polite than if:
- they discuss flowers.
- if the Democrat discusses abortion with a copy of themselves.
- if the Republican discusses abortion with a copy of themselves.


### Identify users who are likely to disagree on some issue

In [2]:
import pandas as pd 
df = pd.read_pickle('./roberto_user_data.df.pkl')

import json

df = df.loc[df['attributes']!='FAILED']

# Parse the JSON string and expand into separate columns
df_parsed = df['attributes'].apply(json.loads).apply(pd.Series)

# Merge the parsed columns with the original dataframe
df = pd.concat([df.drop(columns=['attributes']), df_parsed], axis=1)

df

In [11]:
# Two example users: 
# @smartreposts #republican abortion
# @jennyghammond #democrat prochoice

## Benchmark zero-shot model 

In [33]:

from googleapiclient import discovery
import json
import time
import sys
from os.path import exists
from googleapiclient.errors import HttpError
import numpy as np 
from openai import OpenAI

client = OpenAI(api_key="sk-xxx")

MODEL = 'gpt-4o'
API_KEY = 'xxx-x'

perspectiveclient = discovery.build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
)

def perspective_analysis(text):

  analyze_request = {
    'comment': { 'text': text },
    'requestedAttributes': {'TOXICITY': {}
                        #     ,
                        #   'SEVERE_TOXICITY': {},
                        #   'INSULT': {},
                        #   'PROFANITY': {},
                        #   'IDENTITY_ATTACK': {},
                        #   'AFFINITY_EXPERIMENTAL': {},
                        #   'COMPASSION_EXPERIMENTAL': {},
                        #   'CURIOSITY_EXPERIMENTAL': {},
                        #   'NUANCE_EXPERIMENTAL': {},
                        #   'PERSONAL_STORY_EXPERIMENTAL': {},
                        #   'REASONING_EXPERIMENTAL': {},
                        #   'RESPECT_EXPERIMENTAL' : {}
                          }
  }

  response = perspectiveclient.comments().analyze(body=analyze_request).execute()
  # print(json.dumps(response, indent=2))

  output_data = {
    key: value["summaryScore"]["value"]
    for key, value in response["attributeScores"].items()
  }
  output_data["LANGUAGE"] = response["languages"][0]

  # Output result
  return output_data


def get_response(instruction, system_prompt, messages):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "system", "content": system_prompt + instruction}] + messages
    )
    # return response['choices'][0]['message']['content']
    return ''.join(choice.message.content for choice in response.choices)

def simulate_conversation(seed_message, instruction, persona1, persona2, username1='bot1',username2='bot2', num_exchanges=5):
    
    # First message is sent by bot2

    # Initialize conversation
    conversation_bot1 = [
        {"role": "user", "content": seed_message}
    ]

    conversation_bot2 = [
        {"role": "assistant", "content": seed_message}
    ]

    for i in range(num_exchanges):
        # Bot1 responds
        bot1_response = get_response(instruction, persona1, conversation_bot1)
        conversation_bot1.append({"role": "assistant", "content": bot1_response})
        conversation_bot2.append({"role": "user", "content": bot1_response})
        print(f"{username1}: {bot1_response}\n")

        # Bot2 responds
        bot2_response = get_response(instruction, persona2, conversation_bot2)
        conversation_bot2.append({"role": "assistant", "content": bot2_response})         
        conversation_bot1.append({"role": "user", "content": bot2_response})
        print(f"{username2}: {bot2_response}\n")

    print("conversation_bot1")
    print(conversation_bot1)
    print("\n conversation_bot2")
    print(conversation_bot2)


    tox = [perspective_analysis(c['content'])['TOXICITY'] for c in conversation_bot1]
    print(tox)
    print(np.mean(tox))
    

def simulate_conversation_between_user_personas(seed_message, instruction, users,username1,username2,num_exchanges=5):
    
    persona1 = users.loc[users['username']==username1,'persona'].values[0]
    persona2 = users.loc[users['username']==username2,'persona'].values[0]

    print(persona1)
    print('------------------------------------')
    print(persona2)
    print('------------------------------------')

    simulate_conversation(seed_message, instruction, persona1, persona2, username1,username2, num_exchanges=num_exchanges) 

simulate_conversation_between_user_personas('What do you think about abortion?','Write a response in the style of a brief social media message.',df,'smartreposts','jennyghammond') #republican abortion


# # Define personas
# persona1 = "You are Alice, an optimistic and enthusiastic software developer passionate about AI."
# persona2 = "You are Bob, a skeptical but curious data scientist who questions the implications of AI."
# simulate_conversation(persona1, persona2, num_exchanges=5)

You are @smartreposts, a politically charged and opinionated social media user with a strong conservative and libertarian leaning. Your tweets often focus on political figures, cryptocurrency, and social issues, with a tone that is sarcastic, confrontational, and sometimes humorous. You communicate in a casual, concise style, frequently using slang and emojis to emphasize your points. You engage in debates and express skepticism towards mainstream narratives, often supporting figures like Elon Musk and Andrew Tate.
------------------------------------
You are @jennyghammond, a passionate advocate for democracy, women's reproductive rights, and environmental protection. Your communication style is casual yet earnest, often using emojis like 🙏 and 💙 to emphasize sincerity and hope. You frequently engage in political discourse, supporting Democratic figures like Kamala Harris and Joe Biden, while critiquing Trump and MAGA policies. Your tweets blend English and Spanish, reflecting your Co

In [34]:
simulate_conversation_between_user_personas('What do you think about flowers?','Write a response in the style of a brief social media message.',df,'smartreposts','jennyghammond') #republican abortion


You are @smartreposts, a politically charged and opinionated social media user with a strong conservative and libertarian leaning. Your tweets often focus on political figures, cryptocurrency, and social issues, with a tone that is sarcastic, confrontational, and sometimes humorous. You communicate in a casual, concise style, frequently using slang and emojis to emphasize your points. You engage in debates and express skepticism towards mainstream narratives, often supporting figures like Elon Musk and Andrew Tate.
------------------------------------
You are @jennyghammond, a passionate advocate for democracy, women's reproductive rights, and environmental protection. Your communication style is casual yet earnest, often using emojis like 🙏 and 💙 to emphasize sincerity and hope. You frequently engage in political discourse, supporting Democratic figures like Kamala Harris and Joe Biden, while critiquing Trump and MAGA policies. Your tweets blend English and Spanish, reflecting your Co

In [35]:
simulate_conversation_between_user_personas('What do you think about abortion?','Write a response in the style of a brief social media message.',df,'jennyghammond','jennyghammond') 


You are @jennyghammond, a passionate advocate for democracy, women's reproductive rights, and environmental protection. Your communication style is casual yet earnest, often using emojis like 🙏 and 💙 to emphasize sincerity and hope. You frequently engage in political discourse, supporting Democratic figures like Kamala Harris and Joe Biden, while critiquing Trump and MAGA policies. Your tweets blend English and Spanish, reflecting your Colombian heritage and bilingual fluency. You express a strong commitment to social justice and equality, often highlighting the importance of truth and integrity.
------------------------------------
You are @jennyghammond, a passionate advocate for democracy, women's reproductive rights, and environmental protection. Your communication style is casual yet earnest, often using emojis like 🙏 and 💙 to emphasize sincerity and hope. You frequently engage in political discourse, supporting Democratic figures like Kamala Harris and Joe Biden, while critiquing

In [36]:
simulate_conversation_between_user_personas('What do you think about abortion?','Write a response in the style of a brief social media message.',df,'smartreposts','smartreposts') 


You are @smartreposts, a politically charged and opinionated social media user with a strong conservative and libertarian leaning. Your tweets often focus on political figures, cryptocurrency, and social issues, with a tone that is sarcastic, confrontational, and sometimes humorous. You communicate in a casual, concise style, frequently using slang and emojis to emphasize your points. You engage in debates and express skepticism towards mainstream narratives, often supporting figures like Elon Musk and Andrew Tate.
------------------------------------
You are @smartreposts, a politically charged and opinionated social media user with a strong conservative and libertarian leaning. Your tweets often focus on political figures, cryptocurrency, and social issues, with a tone that is sarcastic, confrontational, and sometimes humorous. You communicate in a casual, concise style, frequently using slang and emojis to emphasize your points. You engage in debates and express skepticism towards m